<a href="https://colab.research.google.com/github/Varshi45/Product-Recommendation-Systems/blob/main/Product_Recommender_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fuzzywuzzy

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from fuzzywuzzy import fuzz, process
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

data = pd.read_csv("/content/drive/MyDrive/Amazon-Products.csv")

# Fill NaN values in 'ratings' and 'no_of_ratings' columns with zeros
data['ratings'].fillna(0, inplace=True)
data['no_of_ratings'].fillna(0, inplace=True)

<ipython-input-12-7e54ad014ee5>:1: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/content/drive/MyDrive/Amazon-Products.csv")


In [ ]:
# Convert 'ratings' column to numeric
data['ratings'] = pd.to_numeric(data['ratings'], errors='coerce')

# Convert 'name' and 'sub_category' columns to strings
data['name'] = data['name'].astype(str)
data['sub_category'] = data['sub_category'].astype(str)

In [ ]:
# Pivot the data to create the ratings utility matrix
ratings_matrix = data.pivot_table(values='ratings', index='name', columns='sub_category', fill_value=0)

# Calculate the cosine similarity between items (sub_categories)
item_similarity = cosine_similarity(ratings_matrix.T)

In [ ]:
# Create a DataFrame with item similarity
item_similarity_df = pd.DataFrame(item_similarity, index=ratings_matrix.columns, columns=ratings_matrix.columns)


In [ ]:
def get_recommendations(product_name, num_recommendations=5, num_products=10):
    # Handle spelling mistakes using fuzzy matching
    best_match, score = process.extractOne(product_name, item_similarity_df.index)

    similar_scores = item_similarity_df[best_match]
    similar_sub_categories = similar_scores.sort_values(ascending=False).index.tolist()

    # Get the exact sub-category that the input product belongs to
    exact_sub_category = process.extractOne(product_name, item_similarity_df.index)[0]

    # Display some products from the exact sub-category
    sub_category_products = data[data['sub_category'] == exact_sub_category].nlargest(num_products, 'ratings')['name']

    # Display top-rated products from the exact sub-category
    print(f"Best rated products from '{exact_sub_category}':")
    for j, product_name in enumerate(sub_category_products):
        print(f"{j+1}: {product_name}")

    # Display top recommended sub-categories
    print(f"\nTop Recommended Sub-Categories for '{product_name}':")
    for i, sub_category in enumerate(similar_sub_categories[:num_recommendations]):
        print(f"{i+1}: {sub_category}")



In [ ]:
# Choose a product name to get recommendations for
while True:
    user_input = input("Enter a product name, sub-category, or 'exit' to quit: ")

    if user_input.lower() == 'exit':
        break

    get_recommendations(user_input, num_recommendations=10, num_products=10)


Enter a product name, sub-category, or 'exit' to quit: headphones
Best rated products from 'Headphones':
1: M10 Wireless Earbuds Bluetooth 5.1 TWS 2200mAh Power Bank Charging Box in Ear Earphones True Stereo Sports Headphones Wate...
2: CLAVIER Pulse in-Ear Headphones/Earphones with Stereo Mic for All Smartphones with Carrying Case (Black)
3: Wireless Earphones for OnePlus Nord/ 1+ /One Plus Nord Original Sports Bluetooth Wireless Earphone with Deep Bass and Neck...
4: SPR™ Compatible for Oppo A5s, Oppo F11 Pro, Oppo F11, Oppo K1, Oppo A9, Oppo F9 Pro, Oppo A3s, Oppo A7, Oppo F11 Pro, Oppo...
5: (Renewed) boAt Rockerz 450 On-Ear Headphones with 15 Hours Battery, 40mm Drivers, Padded Ear Cushions, Easy Access Control...
6: WISHMECHSTORE Wireless Headset S109 Bluetooth v5.0 Ear Clip 16 Hours of Calling with 1 Hour Charge for Music,Calling,Sport...
7: Ckroar M10 True Wireless Earbuds, Bluetooth 5.1 Earphones Auto Pairing Bluetooth TWS Bluetooth Headset (Black, True Wireless)
8: B11 Wirele

In [ ]:
# Save the ratings matrix
with open('ratings_matrix.pkl', 'wb') as f:
    pickle.dump(ratings_matrix, f)

In [ ]:
with open('item_similarity.pkl', 'wb') as f:
    pickle.dump(item_similarity_df, f)

In [ ]:
with open('get_recommendations.pkl', 'wb') as f:
    pickle.dump(get_recommendations, f)